# Retrieval-Augmented Generation                       

### Import Libraries

In [ ]:
import os
import openai
import langchain
from fastapi import HTTPException
from langchain.llms import AzureOpenAI
from langchain.vectorstores import FAISS
from langchain.text_splitter import NLTKTextSplitter
from langchain.embedding.model import OpenAIEmbeddings
from langchain.chains.question_answering import load_qa_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import pyPDFLoader, DirectoryLoader
from langchain.memory import ChatMessageHistory, ConversationBufferMemory
from langchain.chains import LLMChain, RetrievalQA, ConversationalRetrievalChain,RetrievalQAWithSourcesChain
from db.database import engine, SessionLocal
from sqlalchemy.orm import Session
from typing import Annotated
from fastapi import Depends
from pydantic import BaseModel

### Read the external Documents

In [ ]:
data_path = "/Articles/"
try:
    loader = DirectoryLoader(data_path, glob = "**/*.pdf", loader_cls= PyPDFLoader)
    documents = loader.load()
except:
    raise HTTPException(status_code = 400, detail = 'Unable to find the documents required for the vector')

### Create a external Documents embedding

In [1]:
embedding_model_instance = OpenAIEmbeddings(
    model = "text-embedding-ada-002",
    deployment = "***********",
    openai_api_version = "**************************",
    openai_api_base = "************************",
    openai_api_type = "azure",
    openai_proxy = '',
    embedding_ctx_length = 8191,
    openai_api_key = "*****************************",
    openai_organization = '',
    allowed_special = set(),
    disallowed_special = 'all',
    chunk_size = 512,
    max_retries = 6,
    request_timeout = None,
    headers = None,
    tiktoken_model_name = None,
    show_progress_bar = False,
    model_kwargs = {},
    skip_empty = False
    
)

NameError: name 'OpenAIEmbeddings' is not defined

In [ ]:
texts = text_splitter.split_documents(documents)
vectordb = FAISS.from_documents(documents = texts, embedding = embedding_model_instance)
vectordb = save_local("VectorDb")

### Augument the LLM

In [ ]:
llm = AzureOpenAI(deployment_name= "****************", model_name = 'text-embedding-ada-002', temperature = 1)
vectordb = FAISS.load_local("VectordDb")
retriever = vectordb.as_retriever(search_type="mmr", search_kwargs={'k':10})
message_history = ChatMessageHistory()
memory = ConversationBufferMemory(memory_key="chat_history",output_key="answer",chat_memory=message_history,return_message=True)
qa_chain = ConversationalRetrievalChain.from_llm(llm=llm,chain_type="stuff",retriever=retriever,memory=memory,return_source_documents=True)


### Pass the query and fetch the results

In [ ]:
message = "Query"
print(qa_chain(message)['answer'])